In [3]:
# Import dependencies
import pandas as pd
import gmaps
import requests

from config import g_key

ModuleNotFoundError: No module named 'config'

In [4]:
#Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Weather_Database/WeatherPy_database.csv'

In [ ]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your minimum temperature preference? "))
max_temp = float(input("What is your maximum temperature preference? "))

In [ ]:
# Filter the dataset
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

In [ ]:
# Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

In [ ]:
# Drop them if any: there are no any
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

In [ ]:
# Create a DataFrame called hotel_df to store hotel names

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
# Set the parameters for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [ ]:
# Iterature through the Hotel DataFrame
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat}, {lng}"
    
    # Use the search term "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store it
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")

In [ ]:
#new_hotel_df = hotel_df.dropna()
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

In [ ]:
#Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
#Display the figure
fig